# Рекомендация тарифов

Цель: построить модель для классификации пользователей, которая определит наиболее подходящий тариф

План:

1. Общая информация

2. Разделение данных на выборки

3. Исследование моделей

4. Проверка модели на тестовой выборке

5. Проверка модели на адекватность

Решать задачу мы будем с помощью машинного обучения с учителем моделями классификации

## 1. Общая информация

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [ ]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Как видно из общей информации по данным, пропусков нет, типы данных верны. Логично, ведь они уже прошли предобработку, а значит можно приступить к делению на выборки. 

## 2. Разделение данных на выборки

Разделим наш датасет на признаки и целевой признак. Далее из них сделаем 3 выборки: для обучения модели, валидационную и тестовую. Отделим сначала 25% для валидационной выборки, а затем от оставшихся 75% обучающей отделим ещё 25% для теста

In [ ]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=1, stratify=target)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.25, random_state=1, stratify=target_train)

Оценим соотношение размеров выборок

In [ ]:
features_train.shape[0]

1807

In [ ]:
features_valid.shape[0]

804

In [ ]:
features_test.shape[0]

603

Доли размеров train/test/valid примерно 3:1:1, что является правильным соотношением выборок

## 3. Исследование моделей

Сперва попробуем модель решающего дерева

In [ ]:
best_model = None
best_accuracy = 0
best_depth = 0
for depth in range(1, 11):
        model = DecisionTreeClassifier(random_state=1, max_depth=depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy, 'Максимальная глубина:', depth)            

Accuracy наилучшей модели на валидационной выборке: 0.8072139303482587 Максимальная глубина: 10


Как видим, точность модели решающего дерева достаточна для наших целей, однако попробуем использовать много деревьев, построим модель случайного леса 

In [ ]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=1, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_accuracy, 
      'Количество деревьев:', best_est, 'Максимальная глубина:', depth)

Accuracy наилучшей модели на валидационной выборке: 0.8171641791044776 Количество деревьев: 50 Максимальная глубина: 10


Случайный лес показывает результаты получше. Для убедительности посмотрим, как поведёт себя модель логистической регрессии, прежде чем делать выводы

In [ ]:
model = LogisticRegression(random_state=1, solver='liblinear', penalty='l1')
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)

print('Accuracy модели логистической регрессии на валидационной выборке:', accuracy)

Accuracy модели логистической регрессии на валидационной выборке: 0.75


Итак, точность 0.75 впритык устраивает нас, однако есть большая вероятность того, что результаты будух хуже на тестовой выборке, поэтому рисковать с этой моделью не стоит. Будем использовать модель случайного леса, как самую точную

## 4. Проверка модели на тестовой выборке

Проверяем лучшую модель случайного леса (она сохранена в переменной best_model) на тестовой выборке 

In [ ]:
test_accuracy = best_model.score(features_test, target_test)
print(test_accuracy)

0.8092868988391376


Модель показывает хорошие результаты и на тесте, а значит можем перейти к проверке на адекватность

## 5. Проверка модели на адекватность

Проверка на адекватность состоит в том, чтобы использовать неадекватную модель, которая делает предсказания, игнорируя входящие признаки. Если такая модель будет точнее, чем наша, значит наша модель не имеет смысла

In [ ]:
dummy_model = DummyClassifier(random_state=1, strategy='most_frequent')
dummy_model.fit(features_train, target_train)
dummy_accuracy = dummy_model.score(features_valid, target_valid)

print('Accuracy неадеватной модели на валидационной выборке:', dummy_accuracy)

Accuracy неадеватной модели на валидационной выборке: 0.6940298507462687


Проверка показала, что выбранная нами модель случайного леса вполне адекватна и может использоваться для подбора наиболее подходящего пользователю тарифа